In [1]:
import pandas as pd, numpy as np, seaborn as sns, sklearn"C:\Users\Boggu\Downloads\regions.csv"

In [7]:
a = "C:\\Users\Boggu\\Downloads\\customer_nodes.csv"
b = "C:\\Users\\Boggu\\Downloads\\customer_transactions.csv"
c = "C:\\Users\\Boggu\Downloads\\regions.csv"

nodes = pd.read_csv(a)
transactions = pd.read_csv(b)
regions = pd.read_csv(c)


In [19]:
nodes.set_index('Unnamed: 0')

,customer_id,region_id,node_id,start_date,end_date
Unnamed: 0,,,,,
0,1,3,4,2020-01-02,2020-01-03
1,2,3,5,2020-01-03,2020-01-17
2,3,5,4,2020-01-27,2020-02-18
3,4,5,4,2020-01-07,2020-01-19
4,5,3,3,2020-01-15,2020-01-23
...,...,...,...,...,...
3495,496,3,4,2020-02-25,9999-12-31
3496,497,5,4,2020-05-27,9999-12-31
3497,498,1,2,2020-04-05,9999-12-31


In [20]:
nodes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   3500 non-null   int64 
 1   customer_id  3500 non-null   int64 
 2   region_id    3500 non-null   int64 
 3   node_id      3500 non-null   int64 
 4   start_date   3500 non-null   object
 5   end_date     3500 non-null   object
dtypes: int64(4), object(2)
memory usage: 164.2+ KB


In [31]:
transactions.set_index('Unnamed: 0')

,customer_id,txn_date,txn_type,txn_amount
Unnamed: 0,,,,
0,429,2020-01-21,deposit,82
1,155,2020-01-10,deposit,712
2,398,2020-01-01,deposit,196
3,255,2020-01-14,deposit,563
4,185,2020-01-29,deposit,626
...,...,...,...,...
5863,189,2020-02-03,withdrawal,870
5864,189,2020-03-22,purchase,718
5865,189,2020-02-06,purchase,393


In [11]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5868 entries, 0 to 5867
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   5868 non-null   int64 
 1   customer_id  5868 non-null   int64 
 2   txn_date     5868 non-null   object
 3   txn_type     5868 non-null   object
 4   txn_amount   5868 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 229.3+ KB


In [12]:
regions

,Unnamed: 0,region_id,region_name
0,0,1,Australia
1,1,2,America
2,2,3,Africa
3,3,4,Asia
4,4,5,Europe


In [13]:
regions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   5 non-null      int64 
 1   region_id    5 non-null      int64 
 2   region_name  5 non-null      object
dtypes: int64(2), object(1)
memory usage: 252.0+ bytes


# How many unique nodes are there on the Data Bank system?

In [29]:
unique_nodes = nodes['node_id'].nunique()
unique_nodes

5

# What is the number of nodes per region?

In [37]:
number_of_nodes = nodes.groupby('region_id')['node_id'].count()
number_of_nodes

region_id
1    770
2    735
3    714
4    665
5    616
Name: node_id, dtype: int64

# how many customers are allocated to each region

In [44]:
customers_per_region = nodes.groupby('region_id')['customer_id'].count()
customers_per_region

region_id
1    770
2    735
3    714
4    665
5    616
Name: customer_id, dtype: int64

# How many days on average are customers reallocated to a different node?


In [58]:
nodes['start_date'] = pd.to_datetime(nodes['start_date'],errors = 'coerce')
nodes['end_date'] = pd.to_datetime(nodes['end_date'],errors = 'coerce')

In [69]:
nodes['average_days'] = nodes['end_date']- nodes['start_date']
average_days.dropna()
average_days_reallocation = nodes.groupby('customer_id')['average_days'].mean()
average_days_reallocation

customer_id
1     11 days 12:00:00
2     10 days 20:00:00
3     13 days 20:00:00
4     13 days 08:00:00
5      7 days 12:00:00
            ...       
496    7 days 08:00:00
497   18 days 16:00:00
498   11 days 20:00:00
499    4 days 08:00:00
500   14 days 00:00:00
Name: average_days, Length: 500, dtype: timedelta64[ns]

# What is the median, 80th and 95th percentile for this same reallocation days metric for each region?


In [74]:
nodes.groupby('region_id')['average_days'].agg(['median', lambda x : x.quantile(0.8), lambda x : x.quantile(0.95)])

,median,<lambda_0>,<lambda_1>
region_id,,,
1,12 days,23 days,28 days
2,12 days,23 days,28 days
3,12 days,24 days,28 days
4,12 days,23 days,28 days
5,12 days,24 days,28 days


In [85]:
df = pd.merge(nodes, transactions, on = 'customer_id')
df = df.drop(['Unnamed: 0_x','Unnamed: 0_y'],axis = 1)
df

,customer_id,region_id,node_id,start_date,end_date,average_days,txn_date,txn_type,txn_amount
0,1,3,4,2020-01-02,2020-01-03,1 days,2020-01-02,deposit,312
1,1,3,4,2020-01-02,2020-01-03,1 days,2020-03-05,purchase,612
2,1,3,4,2020-01-02,2020-01-03,1 days,2020-03-17,deposit,324
3,1,3,4,2020-01-02,2020-01-03,1 days,2020-03-19,purchase,664
4,1,3,4,2020-01-04,2020-01-14,10 days,2020-01-02,deposit,312
...,...,...,...,...,...,...,...,...,...
41071,500,2,2,2020-04-15,NaT,NaT,2020-01-27,deposit,214
41072,500,2,2,2020-04-15,NaT,NaT,2020-03-01,purchase,929
41073,500,2,2,2020-04-15,NaT,NaT,2020-01-30,deposit,922
41074,500,2,2,2020-04-15,NaT,NaT,2020-01-18,deposit,308


# What is the unique count and total amount for each transaction type?

In [91]:
df['txn_type'].nunique()

3

In [92]:
df.groupby('txn_type')['txn_amount'].sum()

txn_type
deposit       9514176
purchase      5645759
withdrawal    5551021
Name: txn_amount, dtype: int64

# What is the average total historical deposit counts and amounts for all customers?


In [93]:
df

,customer_id,region_id,node_id,start_date,end_date,average_days,txn_date,txn_type,txn_amount
0,1,3,4,2020-01-02,2020-01-03,1 days,2020-01-02,deposit,312
1,1,3,4,2020-01-02,2020-01-03,1 days,2020-03-05,purchase,612
2,1,3,4,2020-01-02,2020-01-03,1 days,2020-03-17,deposit,324
3,1,3,4,2020-01-02,2020-01-03,1 days,2020-03-19,purchase,664
4,1,3,4,2020-01-04,2020-01-14,10 days,2020-01-02,deposit,312
...,...,...,...,...,...,...,...,...,...
41071,500,2,2,2020-04-15,NaT,NaT,2020-01-27,deposit,214
41072,500,2,2,2020-04-15,NaT,NaT,2020-03-01,purchase,929
41073,500,2,2,2020-04-15,NaT,NaT,2020-01-30,deposit,922
41074,500,2,2,2020-04-15,NaT,NaT,2020-01-18,deposit,308


In [100]:
df = df[df['txn_type']=='deposit']
df

,customer_id,region_id,node_id,start_date,end_date,average_days,txn_date,txn_type,txn_amount
0,1,3,4,2020-01-02,2020-01-03,1 days,2020-01-02,deposit,312
2,1,3,4,2020-01-02,2020-01-03,1 days,2020-03-17,deposit,324
4,1,3,4,2020-01-04,2020-01-14,10 days,2020-01-02,deposit,312
6,1,3,4,2020-01-04,2020-01-14,10 days,2020-03-17,deposit,324
8,1,3,2,2020-01-15,2020-01-16,1 days,2020-01-02,deposit,312
...,...,...,...,...,...,...,...,...,...
41068,500,2,2,2020-04-15,NaT,NaT,2020-03-02,deposit,862
41070,500,2,2,2020-04-15,NaT,NaT,2020-02-23,deposit,721
41071,500,2,2,2020-04-15,NaT,NaT,2020-01-27,deposit,214
41073,500,2,2,2020-04-15,NaT,NaT,2020-01-30,deposit,922


In [101]:
avg_deposit_count = df.groupby('customer_id')['txn_amount'].count().mean()
avg_deposit_count

37.394

In [104]:
avg_deposit_amount = df.groupby('customer_id')['txn_amount'].sum().mean()
avg_deposit_amount

19028.352

# What is the closing balance for each customer at the end of the month?

In [108]:
df['amount'] = df.groupby('customer_id')['txn_amount'].cumsum()
closing_balances = df.groupby(['customer_id', df['end_date'].dt.to_period("M")])['amount'].last()
closing_balances

C:\Users\Boggu\AppData\Local\Temp\ipykernel_8064\1979607765.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['amount'] = df.groupby('customer_id')['txn_amount'].cumsum()


customer_id  end_date
1            2020-01      2544
             2020-02      3180
             2020-03      3816
2            2020-01       610
             2020-02      1830
                         ...  
499          2020-02     27102
500          2020-01      6047
             2020-02     18141
             2020-03     30235
             2020-04     36282
Name: amount, Length: 1784, dtype: int64